In [1]:
import pandas as pd

df = pd.read_csv('../data/양파요약데이터_직팜정리.csv', comment='#', encoding='cp949')
df['날짜(YYYY-MM-DD)'] = pd.to_datetime(df['날짜(YYYY-MM-DD)'])
df = df.sort_values('날짜(YYYY-MM-DD)')

# 단가 컬럼 추가
df['단가'] = df['총가격(원)'] / df['단위총물량(kg)']


conda create -n ml-dl-nlp python=3.10 
python.exe -m pip install --upgrade pip 
pip install jupyter numpy==1.23.5 pandas==1.5.3  matplotlib==3.7.0 seaborn==0.12.2 scikit-learn==1.2.1
pip install tensorflow==2.10.0
conda install ipykernel
python -m ipykernel install --user --name ml-dl-nlp --display-name "ml-dl-nlp" 
conda install -c conda-forge imbalanced-learn=0.9.1
pip install xgboost==1.7.3
pip install lightgbm
pip install catboost==1.2